In [1]:
#nos aseguramos de usar solamente ciertas GPUs
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
#import modules
import os
import numpy as np
import tensorflow as tf
import cv2
import pandas as pd
import random
import math
import rasterio
import sys
# import random  
from random import sample 

In [72]:
#import tensorflow libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers

In [ ]:
from sklearn.metrics import mean_squared_error

In [52]:
#read SSI file data
#work with occupancy
df = pd.read_csv('/data/gee/SSI nacional/X_SSI_pp.csv')
ssi = df['pro_ocup_c']


In [24]:
#matlab equivalent to find
def indices(a, func):
    return [i for (i, val) in enumerate(a) if func(val)]

In [21]:
#transform ssi to a list
vi = ssi.values.tolist()

list

In [14]:
tau = 3

In [25]:
#limit value and normalize resulting items
indx_tau = indices(vi, lambda vi: vi  > tau)

In [30]:
type(indx_tau)

list

In [32]:
vi_array = np.array(vi)

In [33]:
vi_array[indx_tau] = tau

In [35]:
vi_array = vi_array/tau


In [36]:
indx_items = indices(vi, lambda vi: (vi>0) & (vi <1)) #limit values for training

In [50]:
vi_selected = vi_array[indx_items] #vulnerability elements of interest

In [39]:
num_imgs = len(indx_items)
print('number of locations = %d' % num_imgs)

number of locations = 630203


In [53]:

lon = df['X_CENT']
lat = df['Y_CENT']
lon_selected = lon[indx_items]
lat_selected = lat[indx_items] 


In [44]:
#read image folders file
df = pd.read_csv('DataSetGeomedianaV2_X_SSI_pp.csv')
folder = df['folder']
img_ini = df['img_ini']
img_end = df['img_end']

#image path
path = '/data/gee/DataSetGeomedianaV2/'

In [48]:
#obtain image filename given the region index
def image_filename(index):
    indf = 0
    for i in np.arange(0,32):
        if (index>=img_ini[i] and index<=img_end[i]):
            indf = i
    img_fn = path + folder[indf] + '/_' + f'{index:07}' + '.tif'
    return img_fn

In [54]:
#classifying ssi with a threshold
n = len(vi_selected)
#th_ssi = 0.2

#ind = np.arange(0,num_imgs)

#pov = np.zeros(num_imgs).astype(int)
#pov[ssi>=th_ssi] = 1

#num_pos = np.sum(ssi>=th_ssi)
#num_neg = np.sum(ssi<th_ssi)

#print(num_pos, num_neg)
#print(num_pos/num_imgs)

In [ ]:
#remove negative samples
seed = 1
random.seed(seed)

#ind_pos = ind[pov==1]
#ind_neg = ind[pov==0]
#ind_neg = random.sample(list(ind_neg), num_pos)

#ind_sel = np.concatenate((ind_pos, ind_neg))
#num_sel = len(ind_sel)

#print(num_sel)

In [63]:
#split indices for training, validation and testing
num_train = np.round(0.5*num_imgs).astype(int)
num_val = np.round(0.25*num_imgs).astype(int)
num_test = num_imgs - num_train - num_val

random.shuffle(indx_items)
ind_train = indx_items[0:num_train]
ind_val = indx_items[num_train:num_train+num_val]
ind_test = indx_items[num_train+num_val:num_imgs]

print(len(ind_train), len(ind_val), len(ind_test))

315102 157551 157550


In [65]:
#test one image
indx = 161400
#channels = [1,2,3]
channels = [1,2,3,4,5,6]
#channels = [4,5,6]
img_fn = image_filename(indx)
image = rasterio.open(img_fn)
img = image.read(channels)
image.close()

print(img_fn)
print(np.shape(img))
print(np.mean(img))

/data/gee/DataSetGeomedianaV2/05_Coahuila/_0161400.tif
(6, 20, 20)
2290.5616666666665


In [66]:
#image size
image_width = 20 
image_height = 20
#rescale intensity values
rescale = 1.0/1.0e4

#load training images
images_train = []
ind_train_valid = []
for indx in ind_train: #for all the training images
    img_fn = image_filename(indx) #get the filename
    if (os.path.isfile(img_fn)):
        image = rasterio.open(img_fn) #open the image
        img = image.read(channels) #read the image
        image.close() #close the image file
        if (np.isnan(np.mean(img))==False):
            sz = np.shape(img) #find out the size of the image
            imr = np.zeros((sz[1],sz[2],sz[0])) #init an tensor with zeros
            for ch in np.arange(0,sz[0]): #copy the image bands
                imr[:,:,ch] = img[ch,:,:]
            small_array = cv2.resize(imr, (image_width, image_height), cv2.INTER_LINEAR) #resize the image, why?
            images_train.append(small_array*rescale) #add it to the output tensor
            ind_train_valid.append(indx) #add the number of image to the index list

print(np.shape(images_train))

(315083, 20, 20, 6)


In [67]:
#load validation images
images_val = [] #init validation tensor
ind_val_valid = [] #init the array containing the number of images
for indx in ind_val:
    img_fn = image_filename(indx)
    if (os.path.isfile(img_fn)):
        image = rasterio.open(img_fn)
        img = image.read(channels)
        image.close()
        if (np.isnan(np.mean(img))==False):
            sz = np.shape(img)
            imr = np.zeros((sz[1],sz[2],sz[0]))
            for ch in np.arange(0,sz[0]):
                imr[:,:,ch] = img[ch,:,:]
            small_array = cv2.resize(imr, (image_width, image_height), cv2.INTER_LINEAR)
            images_val.append(small_array*rescale)
            ind_val_valid.append(indx)

print(np.shape(images_val))

(157544, 20, 20, 6)


In [68]:
#load test images
images_test = []
ind_test_valid = []
for indx in ind_test:
    img_fn = image_filename(indx)
    if (os.path.isfile(img_fn)):
        image = rasterio.open(img_fn)
        img = image.read(channels)
        image.close()
        if (np.isnan(np.mean(img))==False):
            sz = np.shape(img)
            imr = np.zeros((sz[1],sz[2],sz[0]))
            for ch in np.arange(0,sz[0]):
                imr[:,:,ch] = img[ch,:,:]
            small_array = cv2.resize(imr, (image_width, image_height), cv2.INTER_LINEAR)
            images_test.append(small_array*rescale)
            ind_test_valid.append(indx)

print(np.shape(images_test))

(157540, 20, 20, 6)


In [71]:
#input for training the model
x_train = np.array(images_train)
x_val = np.array(images_val)
x_test = np.array(images_test)

y_train = vi_array[ind_train_valid]
y_val = vi_array[ind_val_valid]

print(np.shape(x_train))
print(np.shape(x_val))
print(np.shape(x_test))

(315083, 20, 20, 6)
(157544, 20, 20, 6)
(157540, 20, 20, 6)


In [73]:


sz = np.shape(x_train)

#Instantiate an empty model
model = Sequential()
# C1 Convolutional Layer
model.add(layers.Conv2D(6, kernel_size=(5,5), strides=(1,1),
  activation='tanh', input_shape=(image_width,image_height,sz[3]),
  padding='same'))
# S2 Pooling Layer
model.add(layers.AveragePooling2D(pool_size=(2, 2),
  strides=(1, 1), padding='valid'))
# C3 Convolutional Layer
model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1),
  activation='tanh', padding='valid'))
# S4 Pooling Layer
model.add(layers.AveragePooling2D(pool_size=(2, 2),
  strides=(2, 2), padding='valid'))
# C5 Fully Connected Convolutional Layer
model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1),
  activation='tanh', padding='valid'))
# Flatten the CNN output so that we can connect it with fully connected layers
model.add(layers.Flatten())
# FC6 Fully Connected Layer
model.add(layers.Dense(84, activation='tanh'))
# Output Layer with softmax activation
#model.add(layers.Dense(2, activation='softmax'))
model.add(layers.Dense(1, activation='sigmoid'))

In [76]:
#compile the model
learning_rate = 1e-6
optimizer = tf.keras.optimizers.RMSprop(learning_rate)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
#model.compile(loss=losses.binary_crossentropy, optimizer='SGD', metrics=['accuracy'])

In [ ]:
#train the model
batch_size = 128
step_size_train = len(x_train) / batch_size
step_size_val = len(x_val) / batch_size
epochs = 50
initial_epoch = 0

history = model.fit(x=x_train, y=y_train,
                    steps_per_epoch=step_size_train,
                    validation_data=(x_val, y_val),
                    validation_steps=step_size_val,
                    epochs=epochs,
                    initial_epoch = initial_epoch)

Epoch 1/50
2462/2461 [==============================] - 16s 6ms/step - loss: 0.0020 - mae: 0.0368 - mse: 0.0020 - val_loss: 0.0020 - val_mae: 0.0368 - val_mse: 0.0020
Epoch 2/50
2462/2461 [==============================] - 15s 6ms/step - loss: 0.0020 - mae: 0.0368 - mse: 0.0020 - val_loss: 0.0020 - val_mae: 0.0368 - val_mse: 0.0020
Epoch 3/50
2462/2461 [==============================] - 16s 6ms/step - loss: 0.0020 - mae: 0.0368 - mse: 0.0020 - val_loss: 0.0020 - val_mae: 0.0368 - val_mse: 0.0020
Epoch 4/50
2462/2461 [==============================] - 16s 6ms/step - loss: 0.0020 - mae: 0.0368 - mse: 0.0020 - val_loss: 0.0020 - val_mae: 0.0367 - val_mse: 0.0020
Epoch 5/50
2462/2461 [==============================] - 16s 6ms/step - loss: 0.0020 - mae: 0.0368 - mse: 0.0020 - val_loss: 0.0020 - val_mae: 0.0367 - val_mse: 0.0020
Epoch 6/50
2462/2461 [==============================] - 16s 6ms/step - loss: 0.0020 - mae: 0.0368 - mse: 0.0020 - val_loss: 0.0020 - val_mae: 0.0367 - val_mse: 0.002

In [78]:
loss = history.history['loss']
val_loss = history.history['val_loss']

for k in np.arange(initial_epoch,epochs):
    print('%d,%0.6f,%0.6f' % (k+1, loss[k], val_loss[k]))

1,0.002067,0.002054
2,0.002047,0.002049
3,0.002043,0.002045


In [82]:
#predictions for test images
vi_pred = model.predict(x_test)





In [83]:
vi_real = vi_array[ind_test_valid]
vi_pred = np.array(vi_pred).ravel()

In [86]:


rms = mean_squared_error(vi_real, vi_pred, squared=False)
print(rms)
print(vi_real.std())

0.04511362917481761
0.04814096416215141
